In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.metrics import mean_squared_error,confusion_matrix,f1_score,recall_score,precision_score,mean_absolute_error,r2_score,classification_report,accuracy_score,hamming_loss,zero_one_loss,matthews_corrcoef
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('credit-g.csv')

In [3]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,1000.0,20.903,12.058814,4.0,12.0,18.0,24.00,72.0
credit_amount,1000.0,3271.258,2822.736876,250.0,1365.5,2319.5,3972.25,18424.0
installment_commitment,1000.0,2.973,1.118715,1.0,2.0,3.0,4.00,4.0
residence_since,1000.0,2.845,1.103718,1.0,2.0,3.0,4.00,4.0
age,1000.0,35.546,11.375469,19.0,27.0,33.0,42.00,75.0
existing_credits,1000.0,1.407,0.577654,1.0,1.0,1.0,2.00,4.0
num_dependents,1000.0,1.155,0.362086,1.0,1.0,1.0,1.00,2.0


In [6]:
df.isnull().sum()

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
class                     0
dtype: int64

In [7]:
dosyaAdi = 'credit-g.csv'
veri = pd.read_csv(dosyaAdi)

kategorik_sutunlar = veri.select_dtypes(include=['object'])
veri[kategorik_sutunlar.columns] = veri[kategorik_sutunlar.columns].apply(lambda x: pd.factorize(x)[0])

# Yeni dosya adı ve konumu
yeni_dosya_adresi = os.path.join(os.path.dirname(dosyaAdi), 'num-credit-g.txt')

# Yeni dosyayı oluştur
veri.to_csv(yeni_dosya_adresi, index=False)

In [8]:
def Egitim2Test1Parcala(dosyaAdi):
  f=open(dosyaAdi,"r")
  satirlar=f.readlines()
  basliklar=satirlar[0]
  satirlar=satirlar[1:len(satirlar)]
  random.shuffle(satirlar)
  egitimVerisi=satirlar[0:int(len(satirlar)*2/3)]
  testVerisi=satirlar[int(len(satirlar)*2/3):len(satirlar)]

  dosyaAdi=os.path.basename(dosyaAdi)
  dosyaAdi=os.path.splitext(dosyaAdi)[0]
  f=open(dosyaAdi+"_EgitimVerisi.txt","w")
  f.write(basliklar)
  for satir in egitimVerisi:
    f.write(satir)
  f.close()
  f=open(dosyaAdi+"_TestVerisi.txt","w")
  f.write(basliklar)
  for satir in testVerisi:
    f.write(satir)
  f.close()

Egitim2Test1Parcala("num-credit-g.txt")

In [9]:
def RandomForesttahminEt(egitimVerisetiDosyaAdi,testVerisetiDosyaAdi):
  verisetiEgitim=pd.read_csv(egitimVerisetiDosyaAdi)
  verisetiTest=pd.read_csv(testVerisetiDosyaAdi)
  

  tahminSutunu=verisetiEgitim.columns[len(verisetiEgitim.columns)-1]
  
  ciktiVerisetiEgitim=verisetiEgitim[tahminSutunu]
  verisetiEgitim.drop(labels=[tahminSutunu],axis=1,inplace=True)
  
  ciktiVerisetiTest=verisetiTest[tahminSutunu]
  verisetiTest.drop(labels=[tahminSutunu],axis=1,inplace=True)


  ciktiVerisetiTest=ciktiVerisetiTest.values
  print(verisetiTest.shape)

  verisetiTest=verisetiTest.values
  print(verisetiEgitim.shape)

  verisetiEgitim=verisetiEgitim.values
  ciktiVerisetiEgitim=ciktiVerisetiEgitim.values

  classifier=RandomForestClassifier()
  classifier=classifier.fit(verisetiEgitim,ciktiVerisetiEgitim)
  pre=classifier.predict(verisetiTest)

  cm=confusion_matrix(ciktiVerisetiTest,pre)
  accuracyScore=accuracy_score(ciktiVerisetiTest,pre)

  f1Macro=f1_score(ciktiVerisetiTest,pre,average='macro')
  f1Micro=f1_score(ciktiVerisetiTest,pre,average='micro')

  precisionScore=precision_score(ciktiVerisetiTest,pre,average='macro')
  recalscore=recall_score(ciktiVerisetiTest,pre,average='macro')
    
  hammingLoss=hamming_loss(ciktiVerisetiTest,pre)
  classificatonReport= classification_report(ciktiVerisetiTest,pre)
  zeroOneLoss= zero_one_loss(ciktiVerisetiTest,pre)
  matthewsCorrcoef=matthews_corrcoef(ciktiVerisetiTest,pre)

  cm = confusion_matrix(ciktiVerisetiTest, pre)
  accuracy = accuracy_score(ciktiVerisetiTest, pre)
  f1_macro = f1_score(ciktiVerisetiTest, pre, average='macro')
  f1_micro = f1_score(ciktiVerisetiTest, pre, average='micro')
  precision = precision_score(ciktiVerisetiTest, pre, average='macro')
  recall = recall_score(ciktiVerisetiTest, pre, average='macro')

  print('Accuracy Score: ', accuracy)
  print('Confusion Matrix: \n', cm)
  print('F1 Macro: ', f1_macro)
  print('F1 Micro: ', f1_micro)
  print('Precision Score: ', precision)
  print('Recall Score: ', recall)

  return [cm, accuracy]

RandomForesttahminEt("num-credit-g_EgitimVerisi.txt","num-credit-g_TestVerisi.txt")

(334, 20)
(666, 20)
Accuracy Score:  0.7634730538922155
Confusion Matrix: 
 [[220  22]
 [ 57  35]]
F1 Macro:  0.6587914290517387
F1 Micro:  0.7634730538922155
Precision Score:  0.7041294572170498
Recall Score:  0.6447628458498024


[array([[220,  22],
        [ 57,  35]], dtype=int64),
 0.7634730538922155]

In [10]:
def svm_tahmin_et(egitimVerisetiDosyaAdi, testVerisetiDosyaAdi):
    verisetiEgitim = pd.read_csv(egitimVerisetiDosyaAdi)
    verisetiTest = pd.read_csv(testVerisetiDosyaAdi)

    tahminSutunu = verisetiEgitim.columns[-1]

    ciktiVerisetiEgitim = verisetiEgitim[tahminSutunu]
    verisetiEgitim.drop(labels=[tahminSutunu], axis=1, inplace=True)
    
    ciktiVerisetiTest = verisetiTest[tahminSutunu]
    verisetiTest.drop(labels=[tahminSutunu], axis=1, inplace=True)

    ciktiVerisetiTest = ciktiVerisetiTest.values
    verisetiTest = verisetiTest.values
    verisetiEgitim = verisetiEgitim.values
    ciktiVerisetiEgitim = ciktiVerisetiEgitim.values

    # SVM sınıflandırıcı modeli oluşturma
    classifier = SVC()
    classifier.fit(verisetiEgitim, ciktiVerisetiEgitim)
    predictions = classifier.predict(verisetiTest)

    # Sonuçları değerlendirme
    cm = confusion_matrix(ciktiVerisetiTest, predictions)
    accuracy = accuracy_score(ciktiVerisetiTest, predictions)
    f1_macro = f1_score(ciktiVerisetiTest, predictions, average='macro')
    f1_micro = f1_score(ciktiVerisetiTest, predictions, average='micro')
    precision = precision_score(ciktiVerisetiTest, predictions, average='macro')
    recall = recall_score(ciktiVerisetiTest, predictions, average='macro')

    print('Accuracy Score: ', accuracy)
    print('Confusion Matrix: \n', cm)
    print('F1 Macro: ', f1_macro)
    print('F1 Micro: ', f1_micro)
    print('Precision Score: ', precision)
    print('Recall Score: ', recall)

    return [cm, accuracy]

svm_tahmin_et("num-credit-g_EgitimVerisi.txt", "num-credit-g_TestVerisi.txt")


Accuracy Score:  0.7395209580838323
Confusion Matrix: 
 [[238   4]
 [ 83   9]]
F1 Macro:  0.5084496320730779
F1 Micro:  0.7395209580838323
Precision Score:  0.716870357057273
Recall Score:  0.5406485806683435


[array([[238,   4],
        [ 83,   9]], dtype=int64),
 0.7395209580838323]